In [32]:
import rpy2.ipython.html
from rpy2.robjects import r, globalenv
from rpy2.robjects.packages import importr 

rpy2.ipython.html.init_printing()


In [33]:
rpart = importr("rpart")
rduckdb = importr("duckdb")
rDBI = importr("DBI")


In [34]:
ddb_conn = rDBI.dbConnect(rduckdb.duckdb(), ":memory:")

In [35]:
rDBI.dbExecute(ddb_conn, "SET memory_limit='10GB'")

0.0


In [36]:
pq_path = "/home/mike/workspace/soa-ilec/soa-ilec/data/ilec_2009_19_20210528.parquet"
rDBI.dbExecute(ddb_conn, f"create or replace view ILEC_DATA as select * from read_parquet('{pq_path}')")

0.0


In [37]:
f"create or replace view ILEC_DATA as select * from read_parquet('{pq_path}')"

"create or replace view ILEC_DATA as select * from read_parquet('/home/mike/workspace/soa-ilec/soa-ilec/data/ilec_2009_19_20210528.parquet')"

In [38]:
sql_query = """
select
 Gender,
 Insurance_Plan,
 Smoker_Status,
 Issue_Year,
 sum(Number_Of_Deaths) as Number_Of_Deaths,
 sum(ExpDeathQx2015VBTwMI_byPol) as Expected_Deaths
from ILEC_DATA
group by 
 Gender,
 Insurance_Plan,
 Smoker_Status,
 Issue_Year
"""

df_results = rDBI.dbGetQuery(ddb_conn, sql_query)

In [39]:
robjects.globalenv["df_results"] = df_results

In [67]:
r("""fit_rpart <- rpart(as.matrix(df_results[, c("Expected_Deaths", c("Number_Of_Deaths"))]) ~ Gender + Insurance_Plan, data=df_results, method="poisson")""")
foo = r("""capture.output(print(fit_rpart))""")
print("\n".join(foo))


n= 2627 

node), split, n, deviance, yval
      * denotes terminal node

1) root 2627 83842.090 1.071259  
  2) Insurance_Plan=Other,Perm,Term,ULSG,VL,VLSG 2227 72430.850 1.058837 *
  3) Insurance_Plan=UL 400  4780.366 1.200329 *


[1] "n= 2627 "                                                                  
[2] ""                                                                          
[3] "node), split, n, deviance, yval"                                           
[4] "      * denotes terminal node"                                             
[5] ""                                                                          
[6] "1) root 2627 83842.090 1.071259  "                                         
[7] "  2) Insurance_Plan=Other,Perm,Term,ULSG,VL,VLSG 2227 72430.850 1.058837 *"
[8] "  3) Insurance_Plan=UL 400  4780.366 1.200329 *"                           



In [ ]:
capture_output = r["capture.output"]
foo = capture_output(r.print(rpart_fit)) 


n= 2627 

node), split, n, deviance, yval
      * denotes terminal node

1) root 2627 83842.090 1.071259  
  2) Insurance_Plan=Other,Perm,Term,ULSG,VL,VLSG 2227 72430.850 1.058837 *
  3) Insurance_Plan=UL 400  4780.366 1.200329 *


n= 2627,,"node), split, n, deviance, yval",* denotes terminal node,,1) root 2627 83842.090 1.071259,"2) Insurance_Plan=Other,Perm,Term,ULSG,VL,VLSG 2227 72430.850 1.058837 *",3) Insurance_Plan=UL 400 4780.366 1.200329 *
